<a href="https://colab.research.google.com/github/madesant94/Coursera_Applied_DataScience_Intro/blob/main/RESNET_Process_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pickle as pickle
import pandas as pd
import math
import scipy.io as sio
import argparse
from sklearn import svm
from sklearn.preprocessing import normalize

import os
import time
import pickle
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')


ROOT = "/content/drive/MyDrive/mlp_data/"
os.chdir(f'{ROOT}')

Mounted at /content/drive


In [ ]:
"""import scipy.io as sio
import argparse
import os
import numpy as np
import pandas as pd
import time
import pickle"""

np.random.seed(0)
#Print process
def print_top(dataset_dir, window_size):#, convert, parallel, segment, begin_subject, end_subject, output_dir, set_store):
    print("######################## DEAP EEG data preprocess ########################	\
    		   \n# input directory:	%s \
    		   \n# window size:		%d 	\
    		   \n##############################################################################" % \
          (dataset_dir, window_size))
    return None

#Norm_dataset function
def norm_dataset(dataset_1D):
    print("---------------------------------------------------------------------------------------------------------------")
    print("dataset-1D:",dataset_1D.shape)
    norm_dataset_1D = np.zeros([dataset_1D.shape[0], 32])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_1D[i] = feature_normalize(dataset_1D[i])
    return norm_dataset_1D

#Feature_normalize
def feature_normalize(data):
    mean = data[data.nonzero()].mean()
    sigma = data[data. nonzero ()].std()
    data_normalized = data
    data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
    return data_normalized

#function of getting EEG windon
def windows(data, size):
    start = 0
    while ((start + size) < data.shape[0]):
        yield int(start), int(start + size)
        start += size

#segment_signal_without_transition
def segment_signal_without_transition(data,label,label_index,window_size): #label1,label2,label3,
    print(" label index:", label_index)
    print("data shape:", data.shape)
    for (start, end) in windows(data, window_size):
       # break
        if ((len(data[start:end]) == window_size)):
            if(start == 0):
                segments = data[start:end]
                segments = np.vstack([segments, data[start:end]])
                labels = np.array(label[label_index])
                labels = np.append(labels, np.array(label[label_index]))
                print('*label:', labels)
            else:
                segments = np.vstack([segments, data[start:end]])
                labels = np.append(labels, np.array(label[label_index]))
    print(set(labels))
    #print(set(labels1))
    return segments, labels#,labels1#,labels2,labels3


#Process function
def process(dataset_file, window_size):
    data_file_in = pickle.load(open(file,'rb'), encoding='latin1')
    data_in = data_file_in["data"].transpose(0,2,1) #(40, 8064, 40)(trial channel data)
    #print(data_in)
    #print(' ',data_file_in["labels"][:,1])
    #select label
    #0 valence, 1 arousal, 2 dominance, 3 liking
    
    label_in= (data_file_in["labels"][:,0]>=5).astype(int)*1 # >=5
    label_in1= (data_file_in["labels"][:,1]>=5).astype(int)*10 # >=5
    label_in2= (data_file_in["labels"][:,2]>=5).astype(int)*100 # >=5
    label_in3= (data_file_in["labels"][:,3]>=5).astype(int)*1000 # >=5
    label_in = label_in + label_in1 #+ label_in2 +label_in3
    print('label_in.shape---->',label_in.shape)
    label_inter = []
    label_inter1 = []
    label_inter2 = []
    label_inter3 = []
    data_inter = []
    trials = data_in.shape[0]

    #Process for each trial
    for trial in range(0, trials):
        base_signal = (data_in[trial,0:128,0:32] + data_in[trial,128:256,0:32]+ data_in[trial,256:384,0:32])/3 #(128, 32)
        data = data_in[trial,384:8064,0:32]
        # compute the deviation between baseline signals and experimental signals
        for i in range(0, 60):
            data[i*128:(i+1)*128,0:32] = data[i*128:(i+1)*128,0:32] - base_signal
        label_index = trial
        print("trial:", trial)
        print("data shape:", data.shape)
        data = norm_dataset(data)
        data, label= segment_signal_without_transition(data, label_in,label_index,window_size) #label_in1,label_in2,label_in3
        #,label1,label2,label3
        #rnn
        print('data->',data.shape,'label->',label.shape)
        data = data.reshape(int(data.shape[0]/window_size), window_size, 32)
        data_inter.append(data)
        label_inter.append(label)

    data_inter = np.array(data_inter)
    label_inter = np.array(label_inter)
    print("total data size:", data_inter.shape)
    print("total label size:", label_inter.shape)
    return data_inter,label_inter#,label_inter1#,label_inter2,label_inter3 #,record


dataset_dir = f'{ROOT}DEAP_data_preprocessed_python/' #root of raw data
DATA_DEAP = []
LABEL_DEAP = []
LABEL_DEAP1 = []
LABEL_DEAP2 = []
LABEL_DEAP3 = []


window_size = 128
print(os.listdir(dataset_dir))
#Process for each subjects
record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir, task))]
print(record_list)
for record in record_list:
    file = os.path.join(dataset_dir, record)
    print_top(file, window_size)
    data_inter,label_inter = process(file, window_size) #label_inter1,label_inter2,label_inter3
    DATA_DEAP.append(data_inter)
    LABEL_DEAP.append(label_inter)
    #LABEL_DEAP.append(label_inter1)
    #LABEL_DEAP.append(label_inter2)
    #LABEL_DEAP.append(label_inter3)

DATA_DEAP = np.array(DATA_DEAP) #(32, 40, 60, 128, 32)(subject, trial, second, sampling_point, channel)
print('DATA_DEAP',DATA_DEAP.shape)
DATA_DEAP = np.transpose(DATA_DEAP, (1, 2, 0, 4, 3)) #(40, 60, 32, 32, 128) (trial, second,subject - channel, sampling_point)
print('DATA_DEAP',DATA_DEAP.shape)
LABEL_DEAP = np.array(LABEL_DEAP) #(32, 40, 60)
print('LABEL_DEAP',LABEL_DEAP.shape)
LABEL_DEAP = np.transpose(LABEL_DEAP, (1,2,0)) #(40, 60, 32)
print('LABEL_DEAP_',LABEL_DEAP.shape)
from sklearn.model_selection import train_test_split
DATA_DEAP = np.reshape(DATA_DEAP, (-1,DATA_DEAP.shape[2], 1,DATA_DEAP.shape[-2],DATA_DEAP.shape[-1])) #(2400, 32, 1, 32, 128)
LABEL_DEAP = np.reshape(LABEL_DEAP, (-1, LABEL_DEAP.shape[-1]))#(2400, 32)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
---------------------------------------------------------------------------------------------------------------
dataset-1D: (7680, 32)
 label index: 15
data shape: (7680, 32)
*label: [0 0]
{0}
data-> (7680, 32) label-> (60,)
trial: 16
data shape: (7680, 32)
---------------------------------------------------------------------------------------------------------------
dataset-1D: (7680, 32)
 label index: 16
data shape: (7680, 32)
*label: [1 1]
{1}
data-> (7680, 32) label-> (60,)
trial: 17
data shape: (7680, 32)
---------------------------------------------------------------------------------------------------------------
dataset-1D: (7680, 32)
 label index: 17
data shape: (7680, 32)
*label: [11 11]
{11}
data-> (7680, 32) label-> (60,)
trial: 18
data shape: (7680, 32)
---------------------------------------------------------------------------------------------------------------
dataset-1D: (7680, 32)
 label index: 18
data

In [ ]:
from typing import List, Dict, DefaultDict

G_ = DefaultDict(lambda: 0)
C_ = DefaultDict(lambda: 0)
nn = 0
print(LABEL_DEAP.shape)
for i in range(len(LABEL_DEAP)):
  #print(i,LABEL_DEAP[i])
  a = LABEL_DEAP[i][0:32]
  for j in range(len(a)):
    #G_[(a[j],v[j])] +=1
    #print(i,a[j],C_[(a[j])])
    if a[j] not in C_.keys():
      #print(_.keys())
      C_[a[j]] = nn
      print('. ',a[j], C_[a[j]])
      nn +=1
  #G_[(i)] +=1

print('Total nn',nn)
print(C_.keys())
print('----------------------')


NEW_LABEL_BIN_ = []
for i in range(len(LABEL_DEAP)):
  NEW_LABEL_BIN = []
  a = LABEL_DEAP[i][0:32]
  for j in range(len(a)):
    NEW_LABEL_BIN.append(C_[a[j]])
  NEW_LABEL_BIN_.append(np.array(NEW_LABEL_BIN)) #add label to process

from sklearn.model_selection import train_test_split
x_train_DEAP, x_test_DEAP, y_train_DEAP, y_test_DEAP = train_test_split(DATA_DEAP, NEW_LABEL_BIN_, random_state=42, test_size=0.3)
x_test_DEAP, x_val_DEAP, y_test_DEAP, y_val_DEAP = train_test_split(x_test_DEAP, y_test_DEAP, random_state=42, test_size=0.5)


np.save('x_train_DEAP_4classes.npy',x_train_DEAP)
np.save('x_test_DEAP_4classes.npy',x_test_DEAP)
np.save('x_val_DEAP_4classes.npy',x_val_DEAP)
np.save('y_train_DEAP_4classes.npy',y_train_DEAP)
np.save('y_test_DEAP_4classes.npy',y_test_DEAP)
np.save('y_val_DEAP_4classes.npy',y_val_DEAP)
print('Saved')

(2400, 32)
.  11 0
.  1 1
.  10 2
.  0 3
Total nn 4
dict_keys([11, 1, 10, 0])
----------------------
Saved


In [ ]:
import matplotlib.pyplot as plt
classes = []
counts = []
print('New LABEL',LABEL_DEAP.shape)
for i in sorted(C_.keys()):
  if i in G_.keys():
    #print(i, len(i), i[1])
    counts.append(G_[i]/60)
  else:
    counts.append(0)
  classes.append(str(i).replace('(','').replace(')','').replace(' ',''))
plt.rc('axes', axisbelow=True)
fig = plt.figure(figsize=(12, 6), dpi=80)
#plt.grid()
import matplotlib.cm as cm
from matplotlib.colors import Normalize
my_cmap = cm.get_cmap('YlGnBu')
my_norm = Normalize(vmin=0, vmax=12)
cmap = my_cmap(my_norm(color_n))
#print(len(my_cmap(my_norm(color_n))),my_cmap(my_norm(color_n)))
b = plt.bar(classes,counts, color = cmap , edgecolor = 'black',align='edge', width=1.0)
plt.title('Arousal - Valence 9 Classes Combinations')
plt.xticks(rotation=90)
plt.xlim(-.5,77.5)
plt.ylabel('Counts')
plt.xlabel('Classes')
plt.grid()
colorn = np.arange(0,9,1)
cmap = my_cmap(my_norm(color_n))
cmap_legends = my_cmap(my_norm(colorn))
dict_colors = {}
for i in range(len(cmap_legends)):
  dict_colors['Class '+str(i)] = cmap_legends[i]

handles = [plt.Rectangle((0,0),1,1, color=dict_colors[label]) for label in labels]
plt.legend(handles, labels)

print('my_norm',my_norm)
#spacing = 0.100
#fig.subplots_adjust(bottom=spacing)
plt.show()
fig.savefig('ArousalValence_9Classes_v1.png')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# generate 2 2d grids for the x & y bounds
y, x = np.meshgrid(np.linspace(0,9,10), np.linspace(0, 9, 10))

iz_ = 1
dicto = {'x':[],'y':[],'z':[]}
for i in sorted(G_.keys()):
  #print(i,'x',i[0],'y',i[1],G_[i]/60)
  dicto['x'].append(i[0])
  dicto['y'].append(i[1])
  dicto['z'].append(G_[i]/60)

import pandas as pd

df = pd.DataFrame(dicto)
display(df.head())
import seaborn as sns 
import matplotlib.pyplot as plt

#df = pd.read_clipboard()
table = df.pivot('x', 'y', 'z')
#display(table.head())
fig = plt.figure(figsize=(9, 6), dpi=80)
ax = sns.heatmap(table,cmap='Greys',linewidth=1,linecolor='black') #coolwarm
ax.invert_yaxis()
"""ax.set_ylabel('Arousal')
ax.set_xlabel('Valence')
ax.set_title('Valence - Arousal 9 Classes Combination Heatmap')"""
for _, spine in ax.spines.items():
    spine.set_visible(True)
# 'YlOrRd', 'Wistia','Greys'
plt.ylabel('Arousal')
plt.xlabel('Valence')
plt.title('Valence - Arousal 9 Classes Combination Heatmap')
plt.show()
fig.savefig('heatmap_arousalvalenceV3.png')